In [1]:
import numpy as np
import fitz
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans

/Users/jayeshbhadane/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
def compute_text_embeddings(input_text):
    tokenized_input = bert_tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    model_output = bert_model(**tokenized_input)
    return model_output.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

In [4]:
def summarize_text(input_text):
    sentence_list = input_text.split('. ')
    sentence_embeddings = np.array([compute_text_embeddings(sentence) for sentence in sentence_list])
    cluster_count = min(3, len(sentence_list))
    cluster_model = KMeans(n_clusters=cluster_count)
    cluster_model.fit(sentence_embeddings)
    selected_sentences = []
    
    for cluster_index in range(cluster_count):
        center_of_cluster = cluster_model.cluster_centers_[cluster_index]
        sentences_in_cluster = np.where(cluster_model.labels_ == cluster_index)[0]
        nearest_sentence = min(sentences_in_cluster, key=lambda index: np.linalg.norm(sentence_embeddings[index] - center_of_cluster))
        selected_sentences.append(sentence_list[nearest_sentence])
    return ' '.join(selected_sentences)

In [5]:
content = '''
Lionel Messi, often hailed as one of the greatest footballers of all time, has carved an indelible mark on the world of sports with his exceptional skills, vision, and unprecedented achievements. Born on June 24, 1987, in Rosario, Argentina, Messi's journey to football stardom began at a tender age, when his prodigious talent was evident in his gameplay for his local club, Newell's Old Boys. Despite facing a growth hormone deficiency that threatened to derail his budding career, Messi's relentless determination saw him move to Barcelona at the age of 13, where the club agreed to pay for his medical treatments.

Messi's career at Barcelona, which spanned over two decades, is nothing short of legendary. He made his first-team debut at the age of 17 in 2004 and quickly established himself as one of the most talented players in the world. Over the years, Messi became the backbone of Barcelona, leading the club to numerous victories, including ten La Liga titles, seven Copa del Rey titles, and four UEFA Champions League trophies. His synergy with teammates, particularly during the era of Pep Guardiola's management, propelled Barcelona into one of the most formidable teams in football history.

Messi's individual accolades are equally impressive. He has won the FIFA Ballon d'Or award, given to the world's best player, a record-breaking six times, a testament to his dominance in the sport. Messi's style of play, characterized by his incredible dribbling ability, precise passing, and clinical finishing, has drawn comparisons with football greats like Diego Maradona and Pelé. However, what sets Messi apart is his humility and work ethic, traits that have endeared him to fans and players alike.

Despite his successes with Barcelona, Messi's career was not without its challenges. His loyalty to Barcelona was tested during the 2020-2021 season when financial and administrative turmoil within the club led to his shocking transfer to Paris Saint-Germain (PSG). This move marked a new chapter in Messi's career, offering him the opportunity to prove his prowess in another league and contribute to PSG's quest for their first Champions League trophy.

Messi's influence extends beyond the pitch. He is a global sports icon, with a massive following on social media and numerous endorsement deals with major brands, making him one of the highest-paid athletes in the world. His charitable work, primarily through the Leo Messi Foundation, showcases his commitment to giving back, focusing on health care and education for vulnerable children.

On the international stage, Messi has represented Argentina with distinction. Despite early criticisms of his performance with the national team, he led Argentina to victory in the 2021 Copa America, ending the country's 28-year major trophy drought and cementing his legacy as a national hero.

In conclusion, Lionel Messi's career is a blend of extraordinary talent, unparalleled achievements, and a profound impact on football and society. His journey from a young boy with health challenges to a global football icon is inspiring, reflecting the power of perseverance, dedication, and passion. As Messi continues to write new chapters in his storied career, his legacy as one of the greatest footballers of all time is already secured, serving as an enduring inspiration to aspiring athletes around the world.

'''



summarize_text(content)

/Users/jayeshbhadane/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


"Despite early criticisms of his performance with the national team, he led Argentina to victory in the 2021 Copa America, ending the country's 28-year major trophy drought and cementing his legacy as a national hero.\n\nIn conclusion, Lionel Messi's career is a blend of extraordinary talent, unparalleled achievements, and a profound impact on football and society Born on June 24, 1987, in Rosario, Argentina, Messi's journey to football stardom began at a tender age, when his prodigious talent was evident in his gameplay for his local club, Newell's Old Boys His synergy with teammates, particularly during the era of Pep Guardiola's management, propelled Barcelona into one of the most formidable teams in football history.\n\nMessi's individual accolades are equally impressive"